In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/445/motorbike_data_final.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
fig_dims = (30, 10)
fig, ax = plt.subplots(figsize=fig_dims)
sns.boxplot(x = "Brand", y = "Price", ax=ax, data=df)

In [ ]:
fig_dims = (30, 15)
fig, ax = plt.subplots(figsize=fig_dims)
sns.boxplot(x = "Manufactured_year", y = "Price", ax=ax, data=df)

In [ ]:
class_distribution = df['Brand'].value_counts()
print(class_distribution)
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

# Plot the class distribution
class_distribution.plot(kind='bar')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.show()

## Label encoding


In [ ]:
categorical_columns = ['Brand','Model','Type']
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_enc = LabelEncoder()

# List of categorical columns to be encoded
cate_to_num = categorical_columns

# Apply label encoding to each categorical column and replace the original column with the encoded data
for col in cate_to_num:
    df[col] = label_enc.fit_transform(df[col])

In [ ]:
df.head()

## Outliner remove

In [ ]:
df.boxplot(figsize=(14,5))
plt.title(label='Outliner Distribution',
          fontsize=25,
          color="Red")

In [ ]:
Q1 = df['Distance(km)'].quantile(0.25)
Q3 = df['Distance(km)'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
# Ensure that the lower limit is not less than the minimum value in the data
lower_limit = max(lower_limit, df['Distance(km)'].min())
upper_limit = Q3 + 1.5 * IQR
df['Distance(km)'] = np.where(
   df['Distance(km)'] > upper_limit,
   upper_limit,
   np.where(
       df['Distance(km)'] < lower_limit,
       lower_limit,
       df['Distance(km)']
    )
)

In [ ]:
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
lower_limit = max(lower_limit, df['Distance(km)'].min())
upper_limit = Q3 + 1.5 * IQR
df['Price'] = np.where(
   df['Price'] > upper_limit,
   upper_limit,
   np.where(
       df['Price'] < lower_limit,
       lower_limit,
       df['Price']
    )
)

In [ ]:
Q1 = df['Engine(cc)'].quantile(0.25)
Q3 = df['Engine(cc)'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
df['Engine(cc)'] = np.where(
   df['Engine(cc)'] > upper_limit,
   upper_limit,
   np.where(
       df['Engine(cc)'] < lower_limit,
       lower_limit,
       df['Engine(cc)']
    )
)

In [ ]:
df.boxplot(figsize=(14,5))
plt.title(label='Boxplot after outliner Removing',
          fontsize=25,
          color="Green")

In [ ]:
selected_features = ['Brand','Manufactured_year','Distance(km)','Engine(cc)']
x = df[selected_features]
y = df["Price"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

## ML Model/Algorithm Implementations

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV

#### Create and Train the Linear Regression Model


In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
predictions = lr.predict(x_test)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print('MAE (Mean Absolute Error): %s' %mae)
print('MSE (Mean Squared Error): %s' %mse)
print('RMSE (Root mean squared error): %s' %rmse)
print('R2 score: %s' %r2)

#### Create and Train the Decision Tree Regression Model



In [ ]:
# Define the hyperparameters and their possible values
param_grid = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create the Decision Tree Regressor
regressor = DecisionTreeRegressor(random_state=0)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to your training data
grid_search.fit(x_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model
best_regressor = grid_search.best_estimator_

# Fit the best model to the data
best_regressor.fit(x_train, y_train)

# Now you can use the best model for predictions
y_pred_tr = best_regressor.predict(x_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred_tr)
mse = mean_squared_error(y_test, y_pred_tr)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_tr) *100

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared (R²): {r2}")

In [ ]:
y_pred1=best_regressor.predict([[0,2018,20000,150]])
y_pred1

#### Create and Train the Random Forest Regression Model


In [ ]:
rf_model = RandomForestRegressor(n_estimators = 100, random_state = 50)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred_rf)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_rf) *100

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared (R²): {r2}")

In [ ]:
y_pred1=rf_model.predict([[0,2018,20000,150]])
y_pred1

### Create and Train the KNeighbors Regression Model

In [ ]:
# Define a range of values for k to search over
param_grid = {'n_neighbors': range(3,5)}

# Create a KNN regressor
knn_regressor = KNeighborsRegressor()

# Create a grid search object
grid_search = GridSearchCV(knn_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Get the best value of 'k'
best_k = grid_search.best_params_['n_neighbors']
print(f"The best value of k is {best_k}")

# Create a KNN regressor with the best value of 'k' and train it
best_knn_regressor = KNeighborsRegressor(n_neighbors=best_k)
best_knn_regressor.fit(x_train, y_train)

# Make predictions with the best KNN regressor
y_pred_knn = best_knn_regressor.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred_knn)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_knn)
r2 = r2_score(y_test,y_pred_knn)*100

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared (R²): {r2}")

In [ ]:
import pickle
model_filename = "/content/drive/MyDrive/Colab Notebooks/445/knn_model.pkl"  # Specify the filename for your model
# Save the model to a file
with open(model_filename, 'wb') as file:
    pickle.dump(best_knn_regressor, file)


In [ ]:
y_pred1=best_knn_regressor.predict([[0,2017,46000,150]])
y_pred1

### Create and Train the Gradient Boosting Regression model

In [ ]:
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, max_depth=7, random_state=42)
gb_regressor.fit(x_train, y_train)
y_pred_gb = gb_regressor.predict(x_test)

mse = mean_squared_error(y_test, y_pred_gb)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_gb)
r2 = r2_score(y_test,y_pred_gb)*100

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared (R²): {r2}")

In [ ]:
# Collect all evaluation metrics
metrics = {
    'Model': ['Linear Regression', 'Decision Tree Regressor', 'Random Forest Regressor', 'KNN Regressor', 'Gradient Boosting Regressor'],
    'MAE': [22530.47, 16567.01, 15324.13, 22530.48, 13245.67],  # Replace with actual MAE values
    'MSE': [1450711204.35, 731910584.61, 550784416.89, 1450711204.35, 452345678.12],  # Replace with actual MSE values
    'RMSE': [38088.20, 27053.85, 23464.83, 38088.20, 21263.92],  # Replace with actual RMSE values
    'R² (%)': [69.91, 84.82, 89.34, 69.91, 92.15]  # Replace with actual R² values
}

# Create a DataFrame for better visualization
metrics_df = pd.DataFrame(metrics)

# Display the metrics
print("Model Performance Metrics:")
print(metrics_df)

# Optional: Save the metrics as a CSV file
metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/445/model_metrics_comparison.csv', index=False)

# Optional: Visualize as a bar chart
metrics_df.set_index('Model')[['MAE', 'RMSE']].plot(kind='bar', figsize=(10, 6), title="Model Performance Comparison")
plt.xlabel('Model')
plt.ylabel('Error Values')
plt.xticks(rotation=45)
plt.show()


In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Scale the features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Create SVR model with simple parameters
svr_model = SVR(kernel='rbf', C=100, epsilon=0.1)
svr_model.fit(x_train_scaled, y_train)

# Make predictions
y_pred_svr = svr_model.predict(x_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred_svr)
mse = mean_squared_error(y_test, y_pred_svr)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_svr) * 100

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared (R²): {r2}")

# Example prediction
y_pred1 = svr_model.predict(scaler.transform([[0, 2018, 20000, 150]]))
print("Predicted Price:", y_pred1[0])

# Optional: Save the model
import pickle
model_filename = "/content/drive/MyDrive/Colab Notebooks/445/svm_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(svr_model, file)
